In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
# pd.set_option('display.max_colwidth', None)

In [3]:
pd.reset_option('display.max_colwidth')

In [16]:
collected_08_23 = {
    "otodom_krk" : "../original_data/collected_08_23/master_data/otodom_krk.json",
    "otodom_gda" : "../original_data/collected_08_23/master_data/otodom_gda.json",
    "olx_gda" : "../original_data/collected_08_23/master_data/olx_gda.json",
    "olx_krk" : "../original_data/collected_08_23/master_data/olx_krk.json",
    "gratka_gda" : "../original_data/collected_08_23/master_data/gratka_gda.json",
    "gratak_krk" : "../original_data/collected_08_23/master_data/gratka_krk.json",
    "trojmiasto_gda" : "../original_data/collected_08_23/master_data/trojmiasto_gda.json"
}

# Create a dictionary to store DataFrames
dataframes_08_23 = {}

# Iterate through the JSON data and create DataFrames
for key, value in collected_08_23.items():
    dataframes_08_23[key] = pd.read_json(value)  # Assuming the data is in JSON format

# Now, you have a dictionary of DataFrames where each key is a dataset
# You can access them like this:
otodom_krk_df = dataframes_08_23["otodom_krk"]
otodom_gda_df = dataframes_08_23["otodom_gda"]
olx_gda_df = dataframes_08_23["olx_gda"]
olx_krk_df = dataframes_08_23["olx_krk"]
gratka_gda_df = dataframes_08_23["gratka_gda"]
gratak_krk_df = dataframes_08_23["gratak_krk"]
trojmiasto_gda_df = dataframes_08_23["trojmiasto_gda"]

In [18]:
otodom_gda_df.isna().sum()

link                0
address             3
title               3
price               3
surface             4
rent_extra        242
rooms               4
deposit           309
floor              50
building_type     168
avilable_since    605
outside_are       262
dtype: int64

In [22]:
otodom_gda_df.columns

Index(['link', 'address', 'title', 'price', 'surface', 'rent_extra', 'rooms',
       'deposit', 'floor', 'building_type', 'avilable_since', 'outside_are'],
      dtype='object')

In [23]:
otodom_gda_df["address"].value_counts()

address
Śródmieście, Gdańsk, pomorskie                                 47
ul. Szafarnia, Śródmieście, Gdańsk, pomorskie                  29
ul. Obrońców Wybrzeża, Przymorze Wielkie, Gdańsk, pomorskie    23
ul. Nadmorski Dwór, Brzeźno, Gdańsk, pomorskie                 17
ul. św. Barbary, Śródmieście, Gdańsk, pomorskie                16
                                                               ..
Aniołki, Gdańsk, pomorskie                                      1
Aleja Grunwaldzka, Oliwa, Gdańsk, pomorskie                     1
ul. Nieborowska 46, Chełm, Gdańsk, pomorskie                    1
ul. Słowiańska, Przymorze Małe, Gdańsk, pomorskie               1
ul. Rybaki Dolne, Śródmieście, Gdańsk, pomorskie                1
Name: count, Length: 328, dtype: int64

In [12]:
otodom_gda_df.head()

,rent,location,add_date,Prywatne,Poziom,Umeblowane,Rodzaj zabudowy,Powierzchnia,Liczba pokoi,Czynsz (dodatkowo),Firmowe,negotiable
0,2250.0,Stogi z Przeróbką,18 sierpnia 2023,1.0,2,Tak,Blok,38 m²,2 pokoje,750 zł,NaN,False
1,2700.0,Chełm z dzielnicą Gdańsk Południe,20 sierpnia 2023,1.0,2,Tak,Blok,51 m²,2 pokoje,650 zł,NaN,False
2,2800.0,Siedlce,20 sierpnia 2023,1.0,1,Tak,Apartamentowiec,40 m²,2 pokoje,320 zł,NaN,False
3,2900.0,Przymorze Małe,13 sierpnia 2023,1.0,2,Tak,Apartamentowiec,41 m²,2 pokoje,500 zł,NaN,False
4,350.0,Przymorze Małe,17 sierpnia 2023,1.0,2,Tak,Blok,45 m²,2 pokoje,350 zł,NaN,False


In [10]:
def convert_numeric_to_float(convertable_string):
    return float(convertable_string.replace("zł", "").replace(" ", "").replace("\ndonegocjacji", ""))

def clear_olx(df: pd.DataFrame, city = 'Gdansk'):
    df.drop(
        labels=[
            'link', 
            'username', 
            'on_olx_since', 
            'last_activity',
            'title',
            'description'
        ],
        inplace= True,
        axis= 1
    )
    
    is_negotiable_list = []
    
    for index, row in df.iterrows():

        # negotiable
        if 'do negocjacji' in row['rent']:
            is_negotiable_list.append(True)
        else:
            is_negotiable_list.append(False)
    
    # negotiable apply
    df['negotiable'] = is_negotiable_list


    # rent
    df['rent'] = df['rent'].apply(convert_numeric_to_float)

    # rent_extra
    df = df.rename(mapper={"rent_extra": "Czynsz (dodatkowo)"}, axis=1)
    df['rent_extra'] = df['rent_extra'].apply(convert_numeric_to_float)

In [11]:
otodom_gda_df = clear_olx(otodom_gda_df)

KeyError: 'rent_extra'